In [3]:
import pandas as pd

----
# 보안등현황

In [32]:
df = pd.read_csv("raw_data/9.5개시도_보안등현황.csv")

In [33]:
# year 제거, securitylight_cnt로 변경
df = df.iloc[:,1:]
df["securitylight_cnt"] = df["\nsecuritylight_cnt"]
df = df.iloc[:,[0,4,2,3]]

In [34]:
# 주소에서 na값  확인
df.isnull().sum()

address              10979
securitylight_cnt        0
lon                      0
lat                      0
dtype: int64

In [35]:
df

,address,securitylight_cnt,lon,lat
0,세종특별자치시 금남면 감성리 64-2,1,127.287690,36.443467
1,세종특별자치시 금남면 감성리 267,1,127.288812,36.444181
2,세종특별자치시 금남면 감성리 40-1,1,127.289575,36.444711
3,세종특별자치시 금남면 감성리 26,1,127.290071,36.444455
4,세종특별자치시 금남면 감성리 267,1,127.290002,36.444188
...,...,...,...,...
229400,서울특별시 중랑구 상봉동 19-2,1,127.092434,37.602786
229401,서울특별시 중랑구 상봉동 19-44,1,127.092721,37.602497
229402,서울특별시 중랑구 상봉동 19-25,1,127.092258,37.602374
229403,서울특별시 중랑구 상봉동 495-4,1,127.094617,37.602357


In [36]:
df["관할서"]="모름"

In [37]:
df.head()

,address,securitylight_cnt,lon,lat,관할서
0,세종특별자치시 금남면 감성리 64-2,1,127.287690,36.443467,모름
1,세종특별자치시 금남면 감성리 267,1,127.288812,36.444181,모름
2,세종특별자치시 금남면 감성리 40-1,1,127.289575,36.444711,모름
3,세종특별자치시 금남면 감성리 26,1,127.290071,36.444455,모름
4,세종특별자치시 금남면 감성리 267,1,127.290002,36.444188,모름


----
# 보안등 json

In [38]:
# 보안등 위도 경도 zip
location = list(zip(df["lon"], df["lat"]))

In [39]:
# geopandas, Point  라이브러리
import geopandas as gpd
from shapely.geometry import Point, Polygon

/opt/app-root/lib/python3.6/site-packages/geopandas/_compat.py:110: UserWarning: The Shapely GEOS version (3.8.0-CAPI-1.13.1 ) is incompatible with the GEOS version PyGEOS was compiled with (3.9.0-CAPI-1.16.2). Conversions between both will be slow.
  shapely_geos_version, geos_capi_version_string


In [40]:
#위도 경도 데이터를 Point함수를 이용해서 변수로 만듦
#이를 리스트에 넣어줌
st_list = []
for i in location:
    test_code = Point(i)
    st_list.append(test_code)

In [41]:
#관할서 경계데이터 geopandas로 읽어오기
df_js = gpd.read_file('raw_data/1.5개시도_경찰서_관할경계.geojson', encoding='cp949')
df_js.head()

,NAME,PNAME,geometry
0,세종경찰서,충남청,"MULTIPOLYGON (((127.17202 36.73106, 127.17202 ..."
1,진주경찰서,경남청,"MULTIPOLYGON (((128.26697 35.12927, 128.26697 ..."
2,창원서부경찰서,경남청,"MULTIPOLYGON (((128.63363 35.22152, 128.63357 ..."
3,창원중부경찰서,경남청,"MULTIPOLYGON (((128.60966 35.15093, 128.60956 ..."
4,마산동부경찰서,경남청,"MULTIPOLYGON (((128.62696 35.21714, 128.62695 ..."


In [42]:
# 빠른 반복문 진행을 위해 변수지정
df_js_list = df_js["geometry"]
df_js_name = df_js["NAME"]

In [44]:
from tqdm import tqdm

In [45]:
# 위도경도 데이터가 관할경계데이터에 포함되면  관할서의 이름값을 df관할서의 "모름"을 관할서 이름으로 바꿔라 
# 넘 오래걸림...코드 디벨롶 요망

for z in tqdm(range(41)):
    for i in range(218426):
        if st_list[i].within(df_js_list[z]):
            df.iloc[i,-1] = df.iloc[i,-1].replace("모름",df_js_name[z])
            
        else:
            pass

100%|██████████| 41/41 [1:20:20<00:00, 117.58s/it]


In [48]:
df.to_csv("데이터별_관할서리스트/9_보안등현황_관할서리스트")

In [47]:
df.to_csv("데이터별_관할서리스트/보안등현황_관할서리스트")

In [52]:
df[df["관할서"].str.contains("모름")]

,address,securitylight_cnt,lon,lat,관할서
26391,경상남도 창원시 마산합포구 구산면 구복리 산165-18,1,128.561966,35.065655,모름
26392,경상남도 창원시 마산합포구 구산면 구복리 산165-22,1,128.561722,35.065060,모름
26393,경상남도 창원시 마산합포구 구산면 구복리 산168,1,128.561371,35.064529,모름
26394,경상남도 창원시 마산합포구 구산면 구복리 산165-49,1,128.561859,35.064465,모름
26395,경상남도 창원시 마산합포구 구산면 구복리 산165-49,1,128.562378,35.064430,모름
...,...,...,...,...,...
229400,서울특별시 중랑구 상봉동 19-2,1,127.092434,37.602786,모름
229401,서울특별시 중랑구 상봉동 19-44,1,127.092721,37.602497,모름
229402,서울특별시 중랑구 상봉동 19-25,1,127.092258,37.602374,모름
229403,서울특별시 중랑구 상봉동 495-4,1,127.094617,37.602357,모름


## 3. 보안등현황 재 전처리

#### 3-1. 관할서 처리

In [131]:
df = pd.read_csv("데이터별_관할서리스트/보안등현황_관할서리스트", index_col=0)
df

,address,securitylight_cnt,lon,lat,관할서
0,세종특별자치시 금남면 감성리 64-2,1,127.287690,36.443467,세종경찰서
1,세종특별자치시 금남면 감성리 267,1,127.288812,36.444181,세종경찰서
2,세종특별자치시 금남면 감성리 40-1,1,127.289575,36.444711,세종경찰서
3,세종특별자치시 금남면 감성리 26,1,127.290071,36.444455,세종경찰서
4,세종특별자치시 금남면 감성리 267,1,127.290002,36.444188,세종경찰서
...,...,...,...,...,...
229400,서울특별시 중랑구 상봉동 19-2,1,127.092434,37.602786,모름
229401,서울특별시 중랑구 상봉동 19-44,1,127.092721,37.602497,모름
229402,서울특별시 중랑구 상봉동 19-25,1,127.092258,37.602374,모름
229403,서울특별시 중랑구 상봉동 495-4,1,127.094617,37.602357,모름


In [132]:
df[df["관할서"].str.contains("모")]

,address,securitylight_cnt,lon,lat,관할서
26391,경상남도 창원시 마산합포구 구산면 구복리 산165-18,1,128.561966,35.065655,모름
26392,경상남도 창원시 마산합포구 구산면 구복리 산165-22,1,128.561722,35.065060,모름
26393,경상남도 창원시 마산합포구 구산면 구복리 산168,1,128.561371,35.064529,모름
26394,경상남도 창원시 마산합포구 구산면 구복리 산165-49,1,128.561859,35.064465,모름
26395,경상남도 창원시 마산합포구 구산면 구복리 산165-49,1,128.562378,35.064430,모름
...,...,...,...,...,...
229400,서울특별시 중랑구 상봉동 19-2,1,127.092434,37.602786,모름
229401,서울특별시 중랑구 상봉동 19-44,1,127.092721,37.602497,모름
229402,서울특별시 중랑구 상봉동 19-25,1,127.092258,37.602374,모름
229403,서울특별시 중랑구 상봉동 495-4,1,127.094617,37.602357,모름


In [133]:
df_ch = df[df["관할서"].str.contains("모")]

In [134]:
for i in list(df_ch.index):
    
    if ("구산면") in df_ch["address"][i]:
        df_ch.loc[i,"관할서"] = df_ch.loc[i,"관할서"].replace("모름","마산중부경찰서")
        
    elif ("강동") in df_ch["address"][i]:
        df_ch.loc[i,"관할서"] = df_ch.loc[i,"관할서"].replace("모름","서울강동경찰서")
        
    elif ("수원") in df_ch["address"][i]:
        df_ch.loc[i,"관할서"] = df_ch.loc[i,"관할서"].replace("모름","수원남부경찰서")
        
    elif ("진관") in df_ch["address"][i]:
        df_ch.loc[i,"관할서"] = df_ch.loc[i,"관할서"].replace("모름","서울은평경찰서")
        
    elif ("불광") in df_ch["address"][i]:
        df_ch.loc[i,"관할서"] = df_ch.loc[i,"관할서"].replace("모름","서울은평경찰서")
        
    elif ("갈현") in df_ch["address"][i]:
        df_ch.loc[i,"관할서"] = df_ch.loc[i,"관할서"].replace("모름","서울은평경찰서")
        
    elif ("역촌") in df_ch["address"][i]:
        df_ch.loc[i,"관할서"] = df_ch.loc[i,"관할서"].replace("모름","서울은평경찰서")
        
    elif ("구산") in df_ch["address"][i]:
        df_ch.loc[i,"관할서"] = df_ch.loc[i,"관할서"].replace("모름","서울은평경찰서")
        
    elif ("대조") in df_ch["address"][i]:
        df_ch.loc[i,"관할서"] = df_ch.loc[i,"관할서"].replace("모름","서울은평경찰서")
    
    elif ("중랑구") in df_ch["address"][i]:
        df_ch.loc[i,"관할서"] = df_ch.loc[i,"관할서"].replace("모름","서울중랑경찰서")
        
    elif ("월곡동") in df_ch["address"][i]:
        df_ch.loc[i,"관할서"] = df_ch.loc[i,"관할서"].replace("모름","서울종암경찰서")
        
    elif ("구기동") in df_ch["address"][i]:
        df_ch.loc[i,"관할서"] = df_ch.loc[i,"관할서"].replace("모름","서울종로경찰서")
        
    elif ("평창동") in df_ch["address"][i]:
        df_ch.loc[i,"관할서"] = df_ch.loc[i,"관할서"].replace("모름","서울종로경찰서")
        
    elif ("홍지동") in df_ch["address"][i]:
        df_ch.loc[i,"관할서"] = df_ch.loc[i,"관할서"].replace("모름","서울종로경찰서")
        
    elif ("신영동") in df_ch["address"][i]:
        df_ch.loc[i,"관할서"] = df_ch.loc[i,"관할서"].replace("모름","서울종로경찰서")
        
    elif ("길음") in df_ch["address"][i]:
        df_ch.loc[i,"관할서"] = df_ch.loc[i,"관할서"].replace("모름","서울성북경찰서")
        
    elif ("돈암") in df_ch["address"][i]:
        df_ch.loc[i,"관할서"] = df_ch.loc[i,"관할서"].replace("모름","서울성북경찰서")
        
    elif ("정릉") in df_ch["address"][i]:
        df_ch.loc[i,"관할서"] = df_ch.loc[i,"관할서"].replace("모름","서울성북경찰서")
        
    elif ("동선") in df_ch["address"][i]:
        df_ch.loc[i,"관할서"] = df_ch.loc[i,"관할서"].replace("모름","서울성북경찰서")
        
    elif ("녹번") in df_ch["address"][i]:
        df_ch.loc[i,"관할서"] = df_ch.loc[i,"관할서"].replace("모름","서울서부경찰서")
        
    elif ("신사") in df_ch["address"][i]:
        df_ch.loc[i,"관할서"] = df_ch.loc[i,"관할서"].replace("모름","서울강남경찰서")
        
        
        

/opt/app-root/lib/python3.6/site-packages/pandas/core/indexing.py:1763: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


In [135]:
df_ch[df_ch["관할서"].str.contains("모")]

,address,securitylight_cnt,lon,lat,관할서
50222,서울특별시 강동구 상일동4-10,1,127.182649,37.547590,모름
50226,서울특별시 강동구 상일동4-10,1,127.182670,37.547725,모름
50227,서울특별시 강동구 상일동4-10,1,127.182702,37.547915,모름
50228,서울특별시 강동구 상일동4-12,1,127.182743,37.548366,모름
50229,서울특별시 강동구 상일동12-4,1,127.182873,37.548862,모름
...,...,...,...,...,...
229400,서울특별시 중랑구 상봉동 19-2,1,127.092434,37.602786,모름
229401,서울특별시 중랑구 상봉동 19-44,1,127.092721,37.602497,모름
229402,서울특별시 중랑구 상봉동 19-25,1,127.092258,37.602374,모름
229403,서울특별시 중랑구 상봉동 495-4,1,127.094617,37.602357,모름


In [122]:
df[df["관할서"].str.contains("모")]  = df_ch

In [124]:
df[df["관할서"].str.contains("모")]

,address,securitylight_cnt,lon,lat,관할서


In [130]:
df[df["관할서"].str.contains("동대")]

,address,securitylight_cnt,lon,lat,관할서
136549,서울특별시 성북구 종암동 30-77,1,127.054904,37.591013,서울동대문경찰서


In [125]:
##파일저장
df.to_csv("보안등현황_관할서리스트")

#### 3-2. 본격전처리

In [4]:
df = pd.read_csv("보안등현황_관할서리스트", index_col = 0)
df

,address,securitylight_cnt,lon,lat,관할서
0,세종특별자치시 금남면 감성리 64-2,1,127.287690,36.443467,세종경찰서
1,세종특별자치시 금남면 감성리 267,1,127.288812,36.444181,세종경찰서
2,세종특별자치시 금남면 감성리 40-1,1,127.289575,36.444711,세종경찰서
3,세종특별자치시 금남면 감성리 26,1,127.290071,36.444455,세종경찰서
4,세종특별자치시 금남면 감성리 267,1,127.290002,36.444188,세종경찰서
...,...,...,...,...,...
229400,서울특별시 중랑구 상봉동 19-2,1,127.092434,37.602786,서울중랑경찰서
229401,서울특별시 중랑구 상봉동 19-44,1,127.092721,37.602497,서울중랑경찰서
229402,서울특별시 중랑구 상봉동 19-25,1,127.092258,37.602374,서울중랑경찰서
229403,서울특별시 중랑구 상봉동 495-4,1,127.094617,37.602357,서울중랑경찰서


In [5]:
# 관할서별 cctv개수 합계
df_1 = df.groupby("관할서", sort=True).sum("securitylight_cnt")["securitylight_cnt"]
df_light = pd.DataFrame(df_1).reset_index()
df_light

,관할서,securitylight_cnt
0,마산동부경찰서,5647
1,마산중부경찰서,6843
2,서울강남경찰서,5031
3,서울강동경찰서,7505
4,서울강북경찰서,8430
5,서울강서경찰서,8573
6,서울관악경찰서,11882
7,서울광진경찰서,9132
8,서울구로경찰서,7816
9,서울금천경찰서,6260


In [9]:
# 관할서명  수정
df_light["관할서"] = df_light["관할서"].str.replace("마산", "경남마산")                                     
df_light["관할서"] = df_light["관할서"].str.replace("창원", "경남창원")
df_light["관할서"] = df_light["관할서"].str.replace("수원", "경기수원")
df_light["관할서"] = df_light["관할서"].str.replace("진주", "경남진주")
df_light["관할서"] = df_light["관할서"].str.replace("진해", "경남진해")
df_light["관할서"] = df_light["관할서"].str.replace("세종", "충남세종")                                           

In [38]:
## 동대문경찰서 보안등 1개임 이상함, 양천구 , 송파구는 아예 없음
# 찾아보니 10730개 있음
#동대문 : 10730
#양천구 : 7113
#송파구 : 8817
df_light.loc[20,"securitylight_cnt"]=10730 
new_data = {"관할서":["서울양천경찰서","서울송파경찰서"],"securitylight_cnt":[7113,8817]}
new_df = pd.DataFrame(new_data)
df_light = df_light.append(new_df,ignore_index=True)
df_light

,관할서,securitylight_cnt
0,경기수원남부경찰서,3671
1,경기수원서부경찰서,3961
2,경기수원중부경찰서,6736
3,경남마산동부경찰서,5647
4,경남마산중부경찰서,6843
5,경남진주경찰서,6153
6,경남진해경찰서,3218
7,경남창원서부경찰서,6078
8,경남창원중부경찰서,2895
9,서울강남경찰서,5031


In [39]:
#인덱스 정리
df_light = df_light.sort_values("관할서").reset_index(drop=True)
df_light

,관할서,securitylight_cnt
0,경기수원남부경찰서,3671
1,경기수원서부경찰서,3961
2,경기수원중부경찰서,6736
3,경남마산동부경찰서,5647
4,경남마산중부경찰서,6843
5,경남진주경찰서,6153
6,경남진해경찰서,3218
7,경남창원서부경찰서,6078
8,경남창원중부경찰서,2895
9,서울강남경찰서,5031


In [40]:
##파일저장
df_light.to_csv("전처리완성본/9_관할서별_보안등개수")